In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.2.0-rc3


In [ ]:
playstore = keras.datasets.playstore

# as features
num_words = 20000


(train_data, train_labels), (test_data, test_labels) = dataset.load_data(num_words=num_words)

AttributeError: ignored

The argument num_words=20000 keeps the top 20,000 most frequently occurring words in the training data.

In [ ]:
print("Training examples: {}, labels: {}".format(len(train_data), len(train_labels)))

The text of reviews have been converted to integers, where each integer represents a specific word in a dictionary. Here's what the first review looks like:

In [ ]:
print(train_data[0])

In [ ]:
len(train_data[0]), len(train_data[1])

In [ ]:

max_len = 500

# Convert our lists of integers into 2D tensors
train_data = keras.preprocessing.sequence.pad_sequences(train_data, 
                                                        maxlen=max_len)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, 
                                                       maxlen=max_len)

print(train_data.shape)

In [ ]:
print(train_data[0])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D

embedding_dimension = 16

model = Sequential()
model.add(Embedding(num_words, embedding_dimension, input_length=max_len))


# 'GlobalAveragePooling` 
model.add(GlobalAveragePooling1D())

# classifier 
model.add(Dense(1, activation='sigmoid') )
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

In [ ]:
# word to integer index
word_index = imdb.get_word_index()


word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index[i] for i in text])

In [ ]:
decode_review(train_data[0])

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # 1000, 16. Each word is mapped to an embedding vector.

In [ ]:
out_v = open('vecs.tsv', 'w')
out_m = open('meta.tsv', 'w')
for word_num in range(num_words):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
from google.colab import files
files.download('vecs.tsv')
files.download('meta.tsv')

In [ ]:
# download pretrained GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [ ]:
import os
import numpy as np

In [ ]:
glove_dir = './'

embeddings_index = {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_dim = 100

embedding_matrix = np.zeros((num_words, embedding_dim)) #create an array of zeros with word_num rows and embedding_dim columns
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < num_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
from keras import initializers, models, regularizers
from keras.layers import Dense, Dropout, Embedding, SeparableConv1D, MaxPooling1D, GlobalAveragePooling1D

In [ ]:
#Sequential model
cnnModel = models.Sequential()

#embedding layer
cnnModel.add(Embedding(num_words, 
                    embedding_dim, 
                    input_length=max_len,
                      weights=[embedding_matrix],
                      trainable=False))

Our sepCNN is defined as blocks. Each block will be made up as follows:
* Dropout Layer
* SepConv1D Layer
* SepConv1D Layer
* MaxPooling Layer



In [ ]:
blocks = 4
dropout_rate = 0.3
filters = 50 
kernel_size = 5 
for _ in range(blocks):
    cnnModel.add(Dropout(rate=dropout_rate))
    cnnModel.add(SeparableConv1D(filters=filters,
                             kernel_size= kernel_size,
                             activation= 'relu',
                             bias_initializer= 'random_uniform',
                             depthwise_initializer= 'random_uniform',
                             padding= 'same'))
    cnnModel.add(SeparableConv1D(filters=filters,
                             kernel_size= kernel_size,
                             activation= 'relu',
                             bias_initializer= 'random_uniform',
                             depthwise_initializer= 'random_uniform',
                             padding= 'same'))
    cnnModel.add(MaxPooling1D(pool_size=pool_size))

 adding layers:
* SeparableConv1D
* SeparableConv1D
* GlobalAveragePooling1D
* Dropout
* Dense

In [ ]:
cnnModel.add(SeparableConv1D(filters=filters * 2,
                         kernel_size= kernel_size,
                         activation= 'relu',
                         bias_initializer= 'random_normal',
                         depthwise_initializer= 'random_normal',
                         padding='same'))
cnnModel.add(SeparableConv1D(filters=filters * 2,
                         kernel_size= kernel_size,
                         activation= 'relu',
                         bias_initializer= 'random_normal',
                         depthwise_initializer= 'random_normal',
                         padding='same'))
cnnModel.add(GlobalAveragePooling1D())
cnnModel.add(Dropout(rate= dropout_rate))
cnnModel.add(Dense(1, activation='sigmoid'))

At this point, we have the beginnings of a deep convolutional neural network. We can see the architecture in it's entirity below.

In [ ]:
cnnModel.summary()

train the model and test and validate 

In [ ]:
from keras.optimizers import Adam

In [ ]:
cnnModel.compile(optimizer=Adam(0.1), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = cnnModel.fit(
    train_data,
    train_labels,
    epochs=3,
    batch_size=512,
    validation_split=0.2
)